In [2]:
import numpy as np
import netCDF4 as nc
import calendar

In [23]:
lake_list=['GreatBear','GreatSlave','Athabasca','Winnipegosis', 'Winnipeg','Erie']
lakenum=len(lake_list)
#model='76a'
model='67'
q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfavg_SMALL/wrflsm_d01_daily.nc','r',format='NETCDF4')
#LakeIce= q.variables['IceFrac_A60']#for DM
##for GL25
LAKE_ICEFRAC3D = q.variables['LAKE_ICEFRAC3D']
LakeIce=LAKE_ICEFRAC3D[:,0,:,:] #top layer ice
LakeIce[LakeIce < 0] = 0

#q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfavg_SMALL/wrfsrfc_d01_daily.nc','r',format='NETCDF4')
#LakeIce=q.variables['IceFrac_H0.001']
ntim=len(LakeIce)
nyear=int(ntim/365)
IceFra=np.zeros( (ntim) )
#nyear=int(ntim/365/8)
#IceFra_daily=np.zeros( int(ntim/8) )
IceInOut=np.zeros( (lakenum,4,40) )
IceFraDaily=np.zeros( (lakenum,40,366) )
for lakeid in range(0,3):
    q=nc.Dataset('/project/p/peltier/huoyilin/wmo_basins_shp/'+lake_list[lakeid]+'_mask.nc','r',format='NETCDF4');
    wrf_lake=q.variables['mask_array']
    wrf_lake_sum=np.sum(wrf_lake)
    for itim in range(0, ntim):
        IceFra[itim]= np.sum(LakeIce[itim]*wrf_lake)/wrf_lake_sum
    #for FLAKE wrfsrfc convert 3hrly to daily
#    for itim in range (0,ntim-1,8):
#        IceFra_daily[int(itim/8)] = np.mean(IceFra[itim+1:itim+9])
    #for GL25 &DM wrflsm IceFra is daily
    IceFra_daily = IceFra
    startday=0;
    for iyear in range(0,nyear):
        days_in_year=365 + calendar.isleap(iyear+1979)
        IceFraDaily[lakeid,iyear,0:days_in_year]=IceFra_daily[startday:startday+days_in_year]
        IceInOut[lakeid,0,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] < .9),-999)+1-startday
        IceInOut[lakeid,1,iyear]=next((i for i in range(int(IceInOut[lakeid,0,iyear])+startday,startday+days_in_year) if IceFra_daily[i] < .1),-999)+1-startday
        IceInOut[lakeid,2,iyear]=next((i for i in range(int(IceInOut[lakeid,1,iyear])+startday,startday+days_in_year) if IceFra_daily[i] > .1),-999)+1-startday
        IceInOut[lakeid,3,iyear]=next((i for i in range(int(IceInOut[lakeid,2,iyear])+startday,startday+days_in_year) if IceFra_daily[i] > .9),-999)+1-startday
        startday+=days_in_year
       
        
#model='77a'
model='68'
q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfavg_SMALL/wrflsm_d01_daily.nc','r',format='NETCDF4')
#LakeIce= q.variables['IceFrac_A60']#for DM
##for GL25
LAKE_ICEFRAC3D = q.variables['LAKE_ICEFRAC3D']
LakeIce=LAKE_ICEFRAC3D[:,0,:,:] #top layer ice
LakeIce[LakeIce < 0] = 0

#q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfavg_SMALL/wrfsrfc_d01_daily.nc','r',format='NETCDF4')
#LakeIce=q.variables['IceFrac_H0.001']
ntim=len(LakeIce)
nyear=int(ntim/365)
IceFra=np.zeros( (ntim) )
#nyear=int(ntim/365/8)
#IceFra_daily=np.zeros( int(ntim/8) )
for lakeid in range(3,5):
    q=nc.Dataset('/project/p/peltier/huoyilin/wmo_basins_shp/'+lake_list[lakeid]+'_mask.nc','r',format='NETCDF4');
    wrf_lake=q.variables['mask_array']
    wrf_lake_sum=np.sum(wrf_lake)
    for itim in range(0, ntim):
        IceFra[itim]= np.sum(LakeIce[itim]*wrf_lake)/wrf_lake_sum
    #for FLAKE wrfsrfc convert 3hrly to daily
#    for itim in range (0,ntim-1,8):
#        IceFra_daily[int(itim/8)] = np.mean(IceFra[itim+1:itim+9])
    #for GL25 &DM wrflsm IceFra is daily
    IceFra_daily = IceFra
    startday=0;
    for iyear in range(0,nyear):
        days_in_year=365 + calendar.isleap(iyear+1979)
        IceFraDaily[lakeid,iyear,0:days_in_year]=IceFra_daily[startday:startday+days_in_year]
        IceInOut[lakeid,0,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] < .9),-999)+1-startday
        IceInOut[lakeid,1,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] < .1),-999)+1-startday
        min_index = np.argmin(IceFra_daily[startday:startday+days_in_year])        
        IceInOut[lakeid,2,iyear]=next((i for i in range(min_index+startday,startday+days_in_year) if IceFra_daily[i] > .1),-999)+1-startday
        IceInOut[lakeid,3,iyear]=next((i for i in range(min_index+startday,startday+days_in_year) if IceFra_daily[i] > .9),-999)+1-startday
        startday+=days_in_year        

In [ ]:
lake_list=['GreatBear','GreatSlave','Athabasca','Winnipegosis', 'Winnipeg','Erie']
lakenum=len(lake_list)
IceInOut=np.zeros( (lakenum,4,40) )
IceFraDaily=np.zeros( (lakenum,40,366) )

In [30]:
model='77a'
q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfout/wrfconst_d01.nc','r',format='NETCDF4')
Lakemask=q.variables['LAKEMASK'][0]
#q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfavg_SMALL/wrflsm_d01_daily.nc','r',format='NETCDF4')
#LakeIce= q.variables['IceFrac_A60']#for DM
##for GL25
#LAKE_ICEFRAC3D = q.variables['LAKE_ICEFRAC3D']
#LakeIce=LAKE_ICEFRAC3D[:,0,:,:] #top layer ice
#LakeIce[LakeIce < 0] = 0

q=nc.Dataset('/scratch/p/peltier/mahdinia/ERAI_AND_ERA5_RUN_CASES/WRFTools_RC'+model+'/wrfavg_SMALL/wrfsrfc_d01_daily.nc','r',format='NETCDF4')
LakeIce=q.variables['IceFrac_H0.001']
ntim=len(LakeIce)
nyear=int(ntim/365/8)
#nyear=int(ntim/365)
IceFra=np.zeros( (ntim) )
IceFra_daily=np.zeros( int(ntim/8)  )
for lakeid in range(3,5):
    q=nc.Dataset('/project/p/peltier/huoyilin/wmo_basins_shp/'+lake_list[lakeid]+'_mask.nc','r',format='NETCDF4');
    wrf_lake=q.variables['mask_array']
    wrf_lake_sum=np.sum(wrf_lake*Lakemask)
    for itim in range(0, ntim):
        IceFra[itim]= np.sum(LakeIce[itim]*wrf_lake)/wrf_lake_sum
        #for FLAKE wrfsrfc convert 3hrly to daily
    for itim in range (0,ntim-1,8):
        IceFra_daily[int(itim/8)] = np.mean(IceFra[itim+1:itim+9])
    #for GL25 &DM wrflsm IceFra is daily
    #IceFra_daily = IceFra
    startday=0;
    for iyear in range(0,nyear):
        days_in_year=365 + calendar.isleap(iyear+1979)
        IceFraDaily[lakeid,iyear,0:days_in_year]=IceFra_daily[startday:startday+days_in_year]
        #IceInOut[lakeid,0,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] > .1),-999)+1-startday
        #IceInOut[lakeid,1,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] > .9),-999)+1-startday
        #IceInOut[lakeid,2,iyear]=next((i for i in range(int(IceInOut[lakeid,1,iyear])+startday,startday+days_in_year) if IceFra_daily[i] < .9),-999)+1-startday
        #max_index = np.argmax(IceFra_daily[startday:startday+days_in_year])
        #IceInOut[lakeid,3,iyear]=next((i for i in range(max_index+startday,startday+days_in_year) if IceFra_daily[i] < .1),-999)+1-startday

        IceInOut[lakeid,0,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] < .9),-999)+1-startday
        IceInOut[lakeid,1,iyear]=next((i for i in range(startday,startday+days_in_year) if IceFra_daily[i] < .1),-999)+1-startday
        min_index = np.argmin(IceFra_daily[startday:startday+days_in_year])        
        IceInOut[lakeid,2,iyear]=next((i for i in range(min_index+startday,startday+days_in_year) if IceFra_daily[i] > .1),-999)+1-startday
        IceInOut[lakeid,3,iyear]=next((i for i in range(min_index+startday,startday+days_in_year) if IceFra_daily[i] > .9),-999)+1-startday

        startday+=days_in_year

In [28]:
with open('FLAKEIceInOut.txt', 'w') as outfile:
    lakeid=0
    for slice_2d in IceInOut:
        outfile.write(lake_list[lakeid]+'\n')
        np.savetxt(outfile, slice_2d, fmt='%-8.0f')
        lakeid+=1
        # Writing out a break to indicate different slices...
        outfile.write('\n')
with open('FLAKEIceFraDaily.txt', 'w') as outfile:
    lakeid=0
    for slice_2d in IceFraDaily:
        outfile.write(lake_list[lakeid]+'\n')
        np.savetxt(outfile, slice_2d, fmt='%-8.5f')
        lakeid+=1